In [34]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas

## Spotipy setup

- Need client id and client secret which you can get from [here](https://developer.spotify.com/dashboard/login).
- Also need to setup your redirectURI. I just use Google. It doesn't really matter. You can do this by clicking on the green 'Edit Settings' button. Make sure to add and save.
- Username is your user id which you can find by (at least on the desktop app) clicking on your name in the top right to get to your profile, clicking the '...' under your name, hover over 'Share' and navigate to 'Copy Profile Link'. Click it. Paste somewhere to show the link which should be something like https://open.spotify.com/user/** where ** is your user id.

In [35]:
cId = "xxx"
cSecret = "xxx"
redirectURI = "https://google.com/"
username = "xxx"

client_credentials_manager = SpotifyClientCredentials(client_id=cId, client_secret=cSecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read user-read-currently-playing user-read-playback-state user-read-recently-played'
token = util.prompt_for_user_token(username, scope, cId, cSecret, redirectURI)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Getting user's (your) current track info and putting it into a json file

User needs to at least have spotify open with a song playing or paused. I guess "sessions" are taken into consideration because I tried to run this without having the Spotify app open on my desktop and nothing was being returned.

This is just an example. Not really necessary.

In [36]:
current_track_info = sp.currently_playing()
cp_json_string = json.dumps(current_track_info)
cp_json_data = json.loads(cp_json_string)

# print(json.dumps(current_track_info, sort_keys=False, indent=4))
# print(cp_json_string)
# print(cp_json_data)

with open('current_song.json', 'w') as outfile:
    json.dump(cp_json_data, outfile, sort_keys=True, indent=4)

## Importing some saved tracks from spotify to a csv file

Putting the user's saved tracks into a list named track_list

Notes:
-  The "amount" is for the number of songs we want. I'm getting 100 for now. I have well over 1000 songs saved and it worked fine using that amount. I don't know how this would work if the user does not have more than the amount specified. 

In [37]:
track_list = []

amount = 100

for i in range(50, amount+1, 50):    
    
    saved_tracks = sp.current_user_saved_tracks(limit=50, offset=i)
    
    for t in saved_tracks['items']:
        track = t['track']
        
        track_id = track['id']
        track_name = track['name']
        
        artists = ''
        for a in track['artists']:
            artists += a['name'] + ','
        artists = artists[:-1]
        
        track_album = track['album']
        
        album_name = track_album['name']
        album_img = track_album['images'][0]['url']
        album_type = track_album['album_type']
        release_date = track_album['release_date']
        
        track_dict = {
            'track_id': track_id,
            'track_name': track_name,
            'artists': artists,
            'album_name': album_name,
            'album_img': album_img,
            'album_type': album_type,
            'release_date': release_date
        }
        
        track_list.append(track_dict)

Converting the track list into a dataframe

In [38]:
df = pandas.DataFrame(track_list)
df.set_index('track_id', inplace=True)
df.head()

,album_img,album_name,album_type,artists,release_date,track_name
track_id,,,,,,
0S001mAQoYNpTyVRu8BmMp,https://i.scdn.co/image/8b4e795c17070424342e2b...,Freetown Sound,album,Blood Orange,2016-06-28,Better Than Me
1UdR2sWnDBRd3m0YxZRkMG,https://i.scdn.co/image/8b4e795c17070424342e2b...,Freetown Sound,album,Blood Orange,2016-06-28,Hands Up
5u8ZMevzINPCrN2uXQkeNZ,https://i.scdn.co/image/9942526c238be77c283426...,Cupid Deluxe,album,Blood Orange,2013-11-18,No Right Thing
4Q7Iacj0dnfOu3bdXTcCUA,https://i.scdn.co/image/8b4e795c17070424342e2b...,Freetown Sound,album,Blood Orange,2016-06-28,But You
0e43YnIeghIkDOW146x3G5,https://i.scdn.co/image/8b4e795c17070424342e2b...,Freetown Sound,album,Blood Orange,2016-06-28,E.V.P.


In [39]:
# checking df structure/shape

# 100 rows of data
# 6 columns

df.shape

(100, 6)

Saving the dataframe as a csv

In [40]:
df.to_csv('songs.csv')

Check out the csv file!